In [1]:
from pl_modules import HuggingfaceDatasetWrapper, PlLanguageModelForSequenceOrdering
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForTokenClassification


tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
transformers_model = AutoModelForTokenClassification.from_pretrained(
            'bert-base-cased', return_dict=True, num_labels=1
        )
model = PlLanguageModelForSequenceOrdering(model=transformers_model, target_token_id=tokenizer.cls_token_id)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [2]:
model.configure_optimizers()

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 3e-05
    weight_decay: 0.01
)

In [3]:
from inspect import getfullargspec
getfullargspec(AutoModelForTokenClassification.from_pretrained)

FullArgSpec(args=['cls', 'pretrained_model_name_or_path'], varargs='model_args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [4]:
import torch
import numpy as np
torch.tensor(0) == np.

SyntaxError: invalid syntax (3847689907.py, line 3)

In [ ]:
torch.Tensor

In [9]:
import tensorflow as tf

In [10]:
for summary in tf.train.summary_iterator("):
    print(summary)

AttributeError: module 'tensorflow._api.v2.train' has no attribute 'summary_iterator'

In [12]:
for e in tf.compat.v1.train.summary_iterator('/mnt/data/users/keller/Uni/trf_training_tut/scripts/pytorch-lightning/rocstories/lightning_logs/version_2/events.out.tfevents.1633627405.wcph113.36249.0'):
    print(e)

wall_time: 1633627405.5027034
file_version: "brain.Event:2"

wall_time: 1633627405.5034645
summary {
  value {
    tag: "hp_metric"
    simple_value: -1.0
  }
}

wall_time: 1633627405.5067449
summary {
  value {
    tag: "_hparams_/experiment"
    metadata {
      plugin_data {
        plugin_name: "hparams"
        content: "\022\017*\r\n\013\022\thp_metric"
      }
    }
  }
}

wall_time: 1633627405.5067914
summary {
  value {
    tag: "_hparams_/session_start_info"
    metadata {
      plugin_data {
        plugin_name: "hparams"
        content: "\032\000"
      }
    }
  }
}

wall_time: 1633627405.5068057
summary {
  value {
    tag: "_hparams_/session_end_info"
    metadata {
      plugin_data {
        plugin_name: "hparams"
        content: "\"\002\010\001"
      }
    }
  }
}

wall_time: 1633627412.5747151
summary {
  value {
    tag: "train_loss_step"
    simple_value: 10.162285804748535
  }
}

wall_time: 1633627412.7808573
step: 1
summary {
  value {
    tag: "train_loss_ste

DataLossError: truncated record at 31440' failed with Read less bytes than requested